In [8]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers


In [3]:
from py_isear.isear_loader import IsearLoader
attributes = ['EMOT','SIT']
target = ['TROPHO','TEMPER']
loader = IsearLoader(attributes, target, True)
data = loader.load_isear('path/to/Isear.csv')
data.get_data() # returns attributes
data.get_target() # returns target
data.get_freetext_content() # returns the text content of the database

ModuleNotFoundError: No module named 'py_isear'

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Sample training data
texts = ["This is a positive text.", "Negative sentiment here.", "I feel neutral about this."]
labels = [1, 0, 2]  # 1 for positive, 0 for negative, 2 for neutral

# Tokenize and pad the text data
tokenizer = Tokenizer(num_words=1000, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
sequences = pad_sequences(sequences, maxlen=10, padding='post', truncating='post')

# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=1000, output_dim=16, input_length=10))
model.add(LSTM(32))
model.add(Dense(3, activation='softmax'))  # Assuming you have 3 classes (positive, negative, neutral)

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(sequences, labels, epochs=10, verbose=2)

# Now, you can use the trained model to predict labels for new input text
new_text = ["I like this product."]
new_sequence = tokenizer.texts_to_sequences(new_text)
new_sequence = pad_sequences(new_sequence, maxlen=10, padding='post', truncating='post')
predictions = model.predict(new_sequence)

# Get the predicted label (argmax of the softmax output)
predicted_label = predictions.argmax()
print(f"Predicted label: {predicted_label}")

ValueError: Failed to find data adapter that can handle input: <class 'numpy.ndarray'>, (<class 'list'> containing values of types {"<class 'int'>"})